In [1]:
import pandas as pd
import geopandas as gpd

from libpysal.weights.util import full2W
import warnings
warnings.simplefilter(action='ignore')

import math
import matplotlib.pyplot as plt

from joblib import Parallel, delayed   # <= add this
import numpy as np
import pandas as pd
import statsmodels.api as sm
from esda.moran import Moran

In [2]:
df = pd.read_csv("../00_data/13_final/cbsa_level.csv")
gdf = gpd.read_file("../00_data/01_raw/tl_2024_us_cbsa", engine="pyogrio")
gdf['cbsacode'] = gdf['CBSAFP'].astype('float64')

In [3]:
num_unique = df["cbsacode"].nunique()
print("cbsa:", num_unique)

cbsa: 875


In [4]:
gdf_ll = gdf.to_crs("EPSG:4326").copy()
gdf_ll["centroid"] = gdf_ll.geometry.centroid  # may warn; matches old coordinates exactly
gdf_ll["lon"] = gdf_ll.centroid.x
gdf_ll["lat"] = gdf_ll.centroid.y
gdf_ll["ALAND_acres"] = gdf_ll["ALAND"] / 4046.8564224
df = df.merge(gdf_ll[['ALAND_acres','cbsacode','lon','lat']],how='left')

In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.api as sm
from numpy.linalg import eigh
from esda.moran import Moran
from sklearn.preprocessing import StandardScaler


# ---------------------------
# Safe centroids (projected)
# ---------------------------
def add_lonlat_centroids(gdf, proj_crs="EPSG:5070"):
    """
    Compute centroids in a projected CRS, then bring them back to WGS84.
    """
    gdf_proj = gdf.to_crs(proj_crs).copy()
    cent_proj = gdf_proj.geometry.centroid  # planar centroid in meters
    cent_wgs = gpd.GeoSeries(cent_proj, crs=proj_crs).to_crs("EPSG:4326")
    out = gdf.copy()
    out["centroid"] = cent_wgs
    out["lon"] = cent_wgs.x
    out["lat"] = cent_wgs.y
    return out

# ---------------------------------------
# Fast kNN weights 
# ---------------------------------------

def construct_knn_weights(coords, k):
    # coords: (n,2) [lon, lat] in degrees
    R = 6371.0
    lon = np.radians(coords[:,0])[:,None]  # (n,1)
    lat = np.radians(coords[:,1])[:,None]  # (n,1)

    dlon = lon.T - lon                     # (n,n)
    dlat = lat.T - lat                     # (n,n)

    a = (np.sin(dlat/2.0)**2
         + np.cos(lat) @ np.cos(lat).T * np.sin(dlon/2.0)**2)
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
    dist_mat = R * c
    np.fill_diagonal(dist_mat, np.inf)     # exclude self exactly like before

    n = dist_mat.shape[0]
    W = np.zeros((n,n), dtype=float)
    # identical neighbor selection and weights
    for i in range(n):
        row = dist_mat[i].copy()
        knn_idx = np.argsort(row)[:k]      # same (default quicksort) behavior
        thr = row[knn_idx[-1]]
        if thr > 0:
            W[i, knn_idx] = 1.0 - (dist_mat[i, knn_idx] / thr)

    W_sym = (W + W.T) / 2.0
    row_sums = W_sym.sum(axis=1, keepdims=True)
    nz = row_sums[:,0] > 0
    W_sym[nz] /= row_sums[nz]
    return W_sym

In [6]:
# === Robust Appendix-B forward selection ===
def forward_select_esf(
    eig_for_valid: np.ndarray,        # (n_valid, m_eigs)
    ln_scal_vals: np.ndarray,      # (n_valid,)
    ln_factor_vals: np.ndarray,       # (n_valid,)
    W_lps_sub,                        # PySAL weights for valid rows (full2W)
    alpha: float = 0.05,
    permutations: int = 999,          # Moran permutations; keep 999 for parity
    max_add: int = 100,
    tol_delta_I: float = 1e-6,
):
    """
    Appendix B-style greedy forward selection minimizing Moran's I on residuals.
    Guards against None/constant/NaN eigenvectors and singular fits.
    Stops when p >= alpha, or ΔI is tiny, or max_add reached.
    Returns: (best_model, best_resid, selected_idx)
    """
    import numpy as np
    import statsmodels.api as sm
    from esda.moran import Moran

    n_valid, m_eigs = eig_for_valid.shape
    selected_idx = []

    # Baseline
    Xb = sm.add_constant(ln_scal_vals)
    base = sm.OLS(ln_factor_vals, Xb).fit()
    resid = base.resid
    MI = Moran(resid, W_lps_sub, permutations=permutations)
    if (MI.p_sim >= alpha) or (m_eigs == 0):
        return base, resid, selected_idx

    remaining = list(range(m_eigs))
    best_model = base
    best_resid = resid
    last_I = MI.I

    # Step 1
    best = {"I": np.inf, "p": None, "j": None, "model": None, "resid": None}
    for j in remaining:
        xj = eig_for_valid[:, j]
        if not np.all(np.isfinite(xj)) or np.nanstd(xj) == 0:
            continue
        X = sm.add_constant(np.column_stack([ln_scal_vals, xj]))
        try:
            m = sm.OLS(ln_factor_vals, X).fit()
            r = m.resid
            Mi = Moran(r, W_lps_sub, permutations=permutations)
        except Exception:
            continue
        if (Mi.I < best["I"]) or (np.isclose(Mi.I, best["I"]) and (best["p"] is None or Mi.p_sim > best["p"])):
            best.update({"I": Mi.I, "p": Mi.p_sim, "j": j, "model": m, "resid": r})

    if best["j"] is None:
        return base, resid, selected_idx

    selected_idx.append(best["j"])
    if best["j"] in remaining:
        remaining.remove(best["j"])
    best_model = best["model"]
    best_resid = best["resid"]
    if best["p"] is not None and best["p"] >= alpha:
        return best_model, best_resid, selected_idx

    # Step 2+
    adds = 1
    while remaining and adds < max_add:
        cand = {"I": np.inf, "p": None, "j": None, "model": None, "resid": None}
        for j in remaining:
            X_eigs = eig_for_valid[:, selected_idx + [j]]
            if not np.all(np.isfinite(X_eigs[:, -1])) or np.nanstd(X_eigs[:, -1]) == 0:
                continue
            X = sm.add_constant(np.column_stack([ln_scal_vals, X_eigs]))
            try:
                m = sm.OLS(ln_factor_vals, X).fit()
                r = m.resid
                Mi = Moran(r, W_lps_sub, permutations=permutations)
            except Exception:
                continue
            if (Mi.I < cand["I"]) or (np.isclose(Mi.I, cand["I"]) and (cand["p"] is None or Mi.p_sim > cand["p"])):
                cand.update({"I": Mi.I, "p": Mi.p_sim, "j": j, "model": m, "resid": r})

        if cand["j"] is None:
            break

        # early stop if I barely improves
        if (last_I - cand["I"]) < tol_delta_I:
            break

        selected_idx.append(cand["j"])
        if cand["j"] in remaining:
            remaining.remove(cand["j"])
        best_model = cand["model"]
        best_resid = cand["resid"]
        last_I = cand["I"]
        adds += 1
        if cand["p"] is not None and cand["p"] >= alpha:
            break

    return best_model, best_resid, selected_idx

In [7]:
def calculate_FsAMIs(
    df: pd.DataFrame,
    factor_columns,
    population_column: str,
    cbsa_column: str = "cbsacode",
    alpha: float = 0.05,
    moran_permutations_search: int = 199,
    moran_permutations_final: int = 999,
    n_jobs: int = -1,
    backend: str = "loky",
    k_values = (4, 6, 8, 10, 12, 14, 15, 20, 25, 30),
):
    """
    Stage 1 (per k): choose the single ESF eigenvector that MINIMIZES Moran's I on residuals
                     after adding it to ln(pop). Prefer models with p>=alpha; tie-break on |I| then AIC.
                     The chosen eigenvector index is recorded as 'lead_eig_idx' for that k.
    Stage 2 (at best_k): baseline -> if needed, run your forward selection (which also minimizes Moran's I).
    Returns:
      results_df : per-CBSA FsAMI and model stats at best_k
      k_diag_df  : per-factor per-k diagnostics (lead_eig_idx, I, p, AIC, baseline I/p)
    """
    n_all = len(df)
    print("Merged data rows:", n_all)
    if n_all == 0:
        return pd.DataFrame(), pd.DataFrame()

    # --- numeric copy & coercion ---
    df_num = df.copy()
    for col in [population_column, *factor_columns, "lon", "lat"]:
        if col in df_num.columns:
            df_num[col] = pd.to_numeric(df_num[col], errors="coerce")

    coords_all = df_num[["lon", "lat"]].to_numpy()

    # Precompute W and eigenstuff for all k
    k_cache = {}
    H = np.eye(n_all) - np.ones((n_all, n_all)) / n_all
    for k in k_values:
        W_sym_k = construct_knn_weights(coords_all, k=k)
        Omega = H @ W_sym_k @ H
        evals, evecs = np.linalg.eigh(Omega)
        pos = evals > 1e-12
        evals_pos = evals[pos]
        evecs_pos = evecs[:, pos]
        order = np.argsort(evals_pos)[::-1]  # descending
        k_cache[k] = {
            "W_sym": W_sym_k,
            "evecs": evecs_pos[:, order],
            "evals": evals_pos[order],
        }

    ln_pop_all = np.log(df_num[population_column]).astype(float)

    def _run_one_factor(factor: str):
        rows = []
        diag_rows = []

        fac_vals = df_num[factor]
        pop_vals = df_num[population_column]

        coord_ok = (
            df_num["lon"].notna() & df_num["lat"].notna() &
            np.isfinite(df_num["lon"].to_numpy()) &
            np.isfinite(df_num["lat"].to_numpy())
        )
        valid_mask = (
            fac_vals.notna() & pop_vals.notna() &
            (fac_vals > 0) & (pop_vals > 0) &
            np.isfinite(fac_vals.to_numpy()) &
            np.isfinite(pop_vals.to_numpy()) &
            coord_ok
        )
        if not valid_mask.any():
            print(f"[Skip] No valid data for {factor}")
            return rows, diag_rows

        idx_valid = np.where(valid_mask)[0]
        ln_factor = np.log(fac_vals.loc[valid_mask].to_numpy())
        ln_pop    = ln_pop_all[valid_mask.to_numpy()]
        cbsa_vals = df_num.loc[valid_mask, cbsa_column].to_numpy()

        # ----- Stage 1: for each k, pick the single eigenvector that minimizes Moran's I on residuals -----
        per_k_choice = {}  # k -> dict with chosen eig stats
        for k in k_values:
            cache = k_cache[k]
            W_sub = cache["W_sym"][idx_valid][:, idx_valid]
            W_lps_sub = full2W(W_sub); W_lps_sub.transform = 'r'

            # Baseline (for diagnostics)
            Xb = sm.add_constant(ln_pop)
            model_b = sm.OLS(ln_factor, Xb).fit()
            resid_b = model_b.resid
            MI_b = Moran(resid_b, W_lps_sub, permutations=moran_permutations_search)
            b_I, b_p = float(MI_b.I), float(MI_b.p_sim)

            # Scan all eigenvectors: ln_factor ~ const + ln_pop + e_j
            E = cache["evecs"][idx_valid, :]
            E = E - E.mean(axis=0, keepdims=True)  # column-center
            best = None
            for j in range(E.shape[1]):
                Xj = np.column_stack([np.ones_like(ln_pop), ln_pop, E[:, j]])
                mj = sm.OLS(ln_factor, Xj).fit()
                rj = mj.resid
                MI_j = Moran(rj, W_lps_sub, permutations=moran_permutations_search)
                I_j, p_j = float(MI_j.I), float(MI_j.p_sim)
                aic_j = float(mj.aic)

                cand = {
                    "k": int(k), "eig_idx": int(j),
                    "I": I_j, "p": p_j, "AIC": aic_j,
                    "beta": float(mj.params[1]),
                    "p_beta": float(mj.pvalues[1]),
                }
                # Selection rule within k:
                #   Prefer p>=alpha; among those, minimize |I|, then AIC.
                #   If none reach p>=alpha, maximize p, then minimize |I|, then AIC.
                if best is None:
                    best = cand
                else:
                    def rank_key(d):
                        feas = (d["p"] >= alpha)
                        # negative because we want feasible first (True>False)
                        return (
                            0 if feas else 1,          # feasible first
                            abs(d["I"]) if feas else -d["p"],  # minimize |I| if feasible else maximize p
                            d["AIC"]                   # then AIC
                        )
                    if rank_key(cand) < rank_key(best):
                        best = cand

            # store winner for this k (and log baseline too)
            best.update({"baseline_I": b_I, "baseline_p": b_p})
            per_k_choice[k] = best
            diag_rows.append({
                "factor": factor,
                "k": int(k),
                "lead_eig_idx": int(best["eig_idx"]),
                "lead_I": float(best["I"]),
                "lead_p": float(best["p"]),
                "aic_single": float(best["AIC"]),
                "baseline_I": float(b_I),
                "baseline_p": float(b_p),
            })

        # choose best_k across k:
        #   Prefer p>=alpha; among those, minimize |I|, then AIC, then smaller k
        def across_k_key(d):
            feas = (d["p"] >= alpha)
            return (
                0 if feas else 1,
                abs(d["I"]) if feas else -d["p"],
                d["AIC"],
                d["k"],
            )

        best_k, best_choice = min(
            ((k, info) for k, info in per_k_choice.items()),
            key=lambda kv: across_k_key(kv[1])
        )
        best_k = int(best_k)

        # ----- Stage 2 at best_k: baseline -> forward selection (your function) -----
        cache = k_cache[best_k]
        W_sub = cache["W_sym"][idx_valid][:, idx_valid]
        W_lps_sub = full2W(W_sub); W_lps_sub.transform = 'r'

        # Baseline at best_k (for consistency)
        Xb = sm.add_constant(ln_pop)
        model_b = sm.OLS(ln_factor, Xb).fit()
        resid_b = model_b.resid
        MI_b = Moran(resid_b, W_lps_sub, permutations=moran_permutations_search)
        b_I, b_p = float(MI_b.I), float(MI_b.p_sim)

        best_model = model_b
        best_resid = resid_b
        selected_m = 0

        if b_p < alpha:
            eig_for_valid = cache["evecs"][idx_valid, :]
            eig_for_valid = eig_for_valid - eig_for_valid.mean(axis=0, keepdims=True)
            # Your forward selection is assumed to minimize Moran's I and stop when p>=alpha
            best_model, best_resid, sel_idx = forward_select_esf(
                eig_for_valid=eig_for_valid,
                ln_scal_vals=ln_pop,
                ln_factor_vals=ln_factor,
                W_lps_sub=W_lps_sub,
                alpha=alpha,
                permutations=moran_permutations_search
            )
            selected_m = len(sel_idx)

        # Final Moran at best_k
        MI_f = Moran(best_resid, W_lps_sub, permutations=moran_permutations_final)
        f_I, f_p = float(MI_f.I), float(MI_f.p_sim)

        # Extract stats
        aic_final = float(best_model.aic)
        beta = float(best_model.params[1])
        pval = float(best_model.pvalues[1])
        ci_low, ci_up = map(float, best_model.conf_int(alpha=0.05)[1])

        for i_local, cval in enumerate(cbsa_vals):
            rows.append({
                cbsa_column: cval,
                "factor": factor,
                "best_k": int(best_k),
                "best_k_lead_eig_idx": int(best_choice["eig_idx"]),
                "baseline_moran_value": float(b_I),
                "baseline_moran_p": float(b_p),
                "final_moran_value": float(f_I),
                "final_moran_p": float(f_p),
                "selected_m_eigs": int(selected_m),
                "FsAMI": float(best_resid[i_local]),
                "beta": beta,
                "p_value": pval,
                "CI_lower": ci_low,
                "CI_upper": ci_up,
                "AIC": aic_final
            })
        return rows, diag_rows

    results_lists = Parallel(n_jobs=n_jobs, backend=backend, verbose=0)(
        delayed(_run_one_factor)(factor) for factor in factor_columns
    )

    flat_rows, flat_diag = [], []
    for res_rows, diag_rows in results_lists:
        flat_rows.extend(res_rows or [])
        flat_diag.extend(diag_rows or [])

    results_df = pd.DataFrame(flat_rows)
    k_diag_df  = pd.DataFrame(flat_diag).sort_values(["factor", "k"]).reset_index(drop=True)
    return results_df, k_diag_df

In [8]:
if __name__ == "__main__":
    factor_columns = [
        'BINGE','CSMOKING','DEPRESSION','DIABETES','LPA','OBESITY',
        'adult_smoking','adult_obesity','excessive_drinking','diabetes_prevalence',
        'some_college','unemployment','children_single_parent','mental_health_providers',
        'median_household_income','driving_alone_to_work','sti','FFR20','gdp',
        'coverage_50','coverage_60','coverage_70','coverage_80','coverage_90',
        'noise50n','noise60n','noise70n','noise80n','noise90n','Park_Area_Acres'
    ]

    results_df, k_diag_df = calculate_FsAMIs(   # <= now returns two tables
        df=df,
        factor_columns=factor_columns,
        population_column="TotalPopulation",
        cbsa_column="cbsacode",
        alpha=0.05,
        n_jobs=-1,          # use all cores
        backend="loky",     # safe fork on most platforms
    )

    print("Results (FsAMI):")
    print(results_df.head(10))
    print("\nPer-k diagnostics (chosen eigenvector, Moran's I, AIC):")
    print(k_diag_df.head(10))

    # optional: save
    results_df.to_csv("../00_data/14_output/FsAMI_results_0924.csv", index=False)
    k_diag_df.to_csv("../00_data/14_output/FsAMI_k_diagnostics_0924.csv", index=False)

Merged data rows: 875


/Users/lidouhao/anaconda3/envs/capstone_depression/lib/python3.9/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 3 disconnected components.
 There are 2 islands with ids: 367, 773.
  warnings.warn(message)


('WARNING: ', 367, ' is an island (no neighbors)')
('WARNING: ', 773, ' is an island (no neighbors)')


/Users/lidouhao/anaconda3/envs/capstone_depression/lib/python3.9/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 3 disconnected components.
 There are 2 islands with ids: 367, 773.
  warnings.warn(message)


('WARNING: ', 367, ' is an island (no neighbors)')
('WARNING: ', 773, ' is an island (no neighbors)')


/Users/lidouhao/anaconda3/envs/capstone_depression/lib/python3.9/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
/Users/lidouhao/anaconda3/envs/capstone_depression/lib/python3.9/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)


Results (FsAMI):
   cbsacode factor  best_k  best_k_lead_eig_idx  baseline_moran_value  \
0   10100.0  BINGE      20                    2              0.613429   
1   10140.0  BINGE      20                    2              0.613429   
2   10180.0  BINGE      20                    2              0.613429   
3   10220.0  BINGE      20                    2              0.613429   
4   10300.0  BINGE      20                    2              0.613429   
5   10420.0  BINGE      20                    2              0.613429   
6   10460.0  BINGE      20                    2              0.613429   
7   10500.0  BINGE      20                    2              0.613429   
8   10540.0  BINGE      20                    2              0.613429   
9   10580.0  BINGE      20                    2              0.613429   

   baseline_moran_p  final_moran_value  final_moran_p  selected_m_eigs  \
0             0.005           0.027524          0.018               33   
1             0.005           0